In [5]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
import pandas as pd
import scipy.stats
import math

In [6]:
def getMergedDataFrameFast(typeof):
    truth = pd.read_table(files[typeof]["truth"], delim_whitespace=True, \
       usecols=["transcript_id", "count"])
    df = truth
    df.rename(index=str, \
            columns={"transcript_id": "Name", "count": "count"}, inplace = True)
    
    sla = pd.read_table(files[typeof]["SLA"], delim_whitespace=True, \
                                 usecols=["Name", "NumReads"])
    df = pd.merge(df,sla, how="outer", on = "Name").fillna(0.0)
    
    kallisto = pd.read_table(files[typeof]["kallisto"], delim_whitespace=True, \
                                     usecols=["target_id", "est_counts"])
    kallisto.rename(index=str, columns={"target_id": "Name", \
                                                "est_counts": "NumReads_KAL"}, inplace = True)
    df = pd.merge(df,kallisto, how="outer", on = "Name").fillna(0.0)
    
    hera = pd.read_table(files[typeof]["hera"], delim_whitespace=True, \
                                     usecols=["#target_id", "est_counts"])
    hera["#target_id"]= hera["#target_id"].str.split(":",expand=True)[0]
    hera.rename(index=str, columns={"#target_id": "Name", \
                                                "est_counts": "NumReads_hera"}, inplace = True)
    df = pd.merge(df,hera, how="outer", on = "Name").fillna(0.0)
    return (df,truth,sla,kallisto,hera)

In [7]:
def getMergedDataFrame(typeof):
    df = pd.read_table(files[typeof]["truth"], delim_whitespace=True, \
                                  usecols=["transcript_id", "count"])
    df.rename(index=str, \
            columns={"transcript_id": "Name", "count": "count"}, inplace = True)
    
    sla = pd.read_table(files[typeof]["SLA"], delim_whitespace=True, \
                                 usecols=["Name", "NumReads"])
    df = pd.merge(df,sla, how="outer", on = "Name").fillna(0.0)
    
    kallisto = pd.read_table(files[typeof]["kallisto"], delim_whitespace=True, \
                                     usecols=["target_id", "est_counts"])
    kallisto.rename(index=str, columns={"target_id": "Name", \
                                                "est_counts": "NumReads_KAL"}, inplace = True)
    df = pd.merge(df,kallisto, how="outer", on = "Name").fillna(0.0)
    
    hera = pd.read_table(files[typeof]["hera"], delim_whitespace=True, \
                                     usecols=["#target_id", "est_counts"])
    hera["#target_id"]= hera["#target_id"].str.split(":",expand=True)[0]
    hera.rename(index=str, columns={"#target_id": "Name", \
                                                "est_counts": "NumReads_hera"}, inplace = True)
    df = pd.merge(df,hera, how="outer", on = "Name").fillna(0.0)
    
    star = pd.read_table(files[typeof]["star"], delim_whitespace=True, \
                                 usecols=["Name", "NumReads"])
    star.rename(index = str, columns={"Name": "Name", \
                                              "NumReads": "NumReads_STAR"}, inplace = True)
    
    
    df = pd.merge(df,star, how="outer", on = "Name").fillna(0.0)
                         
    bt = pd.read_table(files[typeof]["bowtie"], delim_whitespace=True, \
                                 usecols=["Name", "NumReads"])
    bt.rename(index = str, columns={"Name": "Name", \
                                              "NumReads": "NumReads_BT"}, inplace = True)
    df = pd.merge(df,bt, how="outer", on = "Name").fillna(0.0)
    return df

In [11]:
prefix = "/mnt/scratch1/bleed_through_human/"
types = ["bleed_through_hera"]
files = {}
for t in types:
    fileinfo = {}
    fileinfo["truth"]= "/".join([prefix,t,"rsem_sim","sim.sim.isoforms.results"])
    fileinfo["SLA"] =  "/".join([prefix,t,"quant","salmon_out","quant.sf"])
    fileinfo["kallisto"] = "/".join([prefix,t,"quant","kallisto_out","abundance.tsv"])
    fileinfo["hera"] = "/".join([prefix,t,"quant","hera_out","abundance.tsv"])
    fileinfo["star"] = "/".join([prefix,t,"quant","star_out","quant.sf"])
    fileinfo["bowtie"] = "/".join([prefix,t,"quant","bowtie_out","quant.sf"])
    files[t] = fileinfo

In [12]:
fileinfo

{'SLA': '/mnt/scratch1/bleed_through_human//bleed_through_hera/quant/salmon_out/quant.sf',
 'bowtie': '/mnt/scratch1/bleed_through_human//bleed_through_hera/quant/bowtie_out/quant.sf',
 'hera': '/mnt/scratch1/bleed_through_human//bleed_through_hera/quant/hera_out/abundance.tsv',
 'kallisto': '/mnt/scratch1/bleed_through_human//bleed_through_hera/quant/kallisto_out/abundance.tsv',
 'star': '/mnt/scratch1/bleed_through_human//bleed_through_hera/quant/star_out/quant.sf',
 'truth': '/mnt/scratch1/bleed_through_human//bleed_through_hera/rsem_sim/sim.sim.isoforms.results'}

In [13]:
df_30 = getMergedDataFrame("bleed_through_hera")

In [17]:
print("kallisto",df_30["count"].corr(df_30["NumReads_KAL"],method="spearman"))
print("SLA",df_30["count"].corr(df_30["NumReads"],method="spearman"))
print("hera",df_30["count"].corr(df_30["NumReads_hera"],method="spearman"))
print("star",df_30["count"].corr(df_30["NumReads_STAR"],method="spearman"))
print("bt",df_30["count"].corr(df_30["NumReads_BT"],method="spearman"))

('kallisto', 0.75366744405042541)
('SLA', 0.842216148287251)
('hera', 0.7572425813340482)
('star', 0.8494506433843485)
('bt', 0.85182281490371081)


In [15]:
df_30['ARD_ratio'] = ((np.abs(df_30["NumReads_BT"] - df_30["NumReads"])+1) \
 /(np.abs(df_30["NumReads_BT"] - df_30["NumReads_hera"])+1)).fillna(0)

In [16]:
df_30.sort_values(['ARD_ratio'],ascending=False)

,Name,count,NumReads,NumReads_KAL,NumReads_hera,NumReads_STAR,NumReads_BT,ARD_ratio
78553,ENST00000541645.1,0.0,402.866550,0.000000e+00,0.000000,0.000000,0.000000,403.866550
93933,ENST00000535788.1,0.0,3342.686263,4.591730e-08,8.226491,11.872627,0.000000,362.400642
54577,ENST00000617614.4,0.0,342.822467,1.402060e+02,0.000000,0.000000,0.000000,343.822467
78555,ENST00000538617.5,0.0,4126.319623,3.118540e+01,16.813542,0.000000,0.000000,231.695618
93932,ENST00000578649.1,0.0,755.349754,6.371660e+00,2.584337,5.407183,0.000000,211.015246
15621,ENST00000617931.4,1408.0,772.161286,1.036510e+03,1321.459416,960.284709,1318.966177,156.818612
78549,ENST00000541272.1,0.0,514.201515,1.781430e+00,3.030208,3.934594,0.000000,127.834969
78554,ENST00000546120.2,0.0,124.821916,0.000000e+00,0.000000,0.000000,0.000000,125.821916
93938,ENST00000578706.5,0.0,99.337432,3.120700e+00,0.000000,0.000000,0.000000,100.337432
78547,ENST00000535131.1,0.0,90.005524,0.000000e+00,0.000000,0.000000,0.000000,91.005524


In [19]:
hera = pd.read_table("/mnt/scratch1/bleed_through_human/bleed_through_hera/quant/hera_out/abundance.tsv",\
                     delim_whitespace=True, \
                                     usecols=["#target_id", "est_counts"])
hera["#target_id"]= hera["#target_id"].str.split(":",expand=True)[0]
hera.rename(index=str, columns={"#target_id": "Name", \
                                                "est_counts": "NumReads_hera"}, inplace = True)

In [ ]:
hera.to_c

In [161]:
print("kallisto",len(df_30[(df_30['count'] == 0) & (df_30['NumReads_KAL'])]))
print("SLA",len(df_30[(df_30['count'] == 0) & (df_30['NumReads'])]))
print("hera",len(df_30[(df_30['count'] == 0) & (df_30['NumReads_hera'])]))
print("star",len(df_30[(df_30['count'] == 0) & (df_30['NumReads_STAR'])]))
print("bt",len(df_30[(df_30['count'] == 0) & (df_30['NumReads_BT'])]))

('kallisto', 6987)
('SLA', 4018)
('hera', 3900)
('star', 3962)
('bt', 3597)


In [281]:
prefix = "/mnt/scratch6/hirak/SLA_benchmark/bleed_through/"
types = ["30_percent"]
files = {}
for t in types:
    fileinfo = {}
    fileinfo["truth"]= "/".join([prefix,t,"rsem_sim","sim.sim.isoforms.results"])
    fileinfo["SLA"] =  "/".join([prefix,t,"quant","salmon_out","quant.sf"])
    fileinfo["kallisto"] = "/".join([prefix,t,"quant","kallisto_out","abundance.tsv"])
    fileinfo["hera"] = "/".join([prefix,t,"quant","hera_out","abundance.tsv"])
    files[t] = fileinfo

In [291]:
df_30, truth, sla,kallisto,hera = getMergedDataFrameFast("30_percent")

In [305]:
set(truth.Name.values).difference(set(sla.Name.values))

{'ENST00000517568.4',
 'ENST00000462877.4',
 'ENST00000457805.5',
 'ENST00000457316.4',
 'ENST00000530454.4',
 'ENST00000556783.4',
 'ENST00000592202.4',
 'ENST00000429662.5',
 'ENST00000322954.9',
 'ENST00000514972.4',
 'ENST00000337665.7',
 'ENST00000383738.5',
 'ENST00000513206.4',
 'ENST00000402239.6',
 'ENST00000418055.4',
 'ENST00000614486.3',
 'ENST00000536290.4',
 'ENST00000351660.8',
 'ENST00000572182.4',
 'ENST00000462224.4',
 'ENST00000505993.4',
 'ENST00000559048.4',
 'ENST00000023064.7',
 'ENST00000466993.4',
 'ENST00000470476.4',
 'ENST00000262764.9',
 'ENST00000389629.7',
 'ENST00000254262.8',
 'ENST00000371217.8',
 'ENST00000521943.4',
 'ENST00000421999.5',
 'ENST00000291416.8',
 'ENST00000417398.4',
 'ENST00000450491.4',
 'ENST00000405816.4',
 'ENST00000511336.4',
 'ENST00000340490.6',
 'ENST00000485326.5',
 'ENST00000392826.5',
 'ENST00000555647.4',
 'ENST00000374555.6',
 'ENST00000535355.4',
 'ENST00000407170.5',
 'ENST00000412265.4',
 'ENST00000468577.4',
 'ENST0000

In [288]:
print("kallisto",df_30["count"].corr(df_30["NumReads_KAL"],method="spearman"))
print("SLA",df_30["count"].corr(df_30["NumReads"],method="spearman"))
print("hera",df_30["count"].corr(df_30["NumReads_hera"],method="spearman"))

('kallisto', 0.1436230151626629)
('SLA', 0.16897107007145784)
('hera', 0.16744798364338395)


In [250]:
salmon_quant_file = "/mnt/scratch4/SLA-benchmarking/clean-benchmarks/selective-alignment-experiment/\
quant_scripts/samples/ERR188140/result.salmon.k31/quant.sf" 

In [251]:
quant = pd.read_table(salmon_quant_file, delim_whitespace=True, \
                                 usecols=["Name", "NumReads"])

In [252]:
quant["Name_clean"] = quant["Name"].str.split(":",expand=True)[0]

In [253]:
tid30 = pd.read_table("/mnt/scratch6/hirak/SLA_benchmark/refpc/tid_60.txt",header=None,names=['tid'])

In [254]:
quant.loc[~quant['Name_clean'].isin(tid30.tid.values),'NumReads'] = 0.0

In [255]:
quant.columns = ['transcript_id','count','to_drop']

In [256]:
quant['count'] = np.ceil(quant['count'])

In [257]:
len(quant[quant['count'] != 0])

33388

In [258]:
len(quant)

199851

In [259]:
to_keep = list(quant['transcript_id'].values)

In [260]:
len(to_keep)

199851

In [261]:
new_tr = pd.read_table("/mnt/scratch4/SLA-benchmarking/clean-benchmarks/selective-alignment-experiment/quant_scripts/salmon.index.31/duplicate_clusters.tsv")

In [262]:
df_keep = pd.DataFrame()
df_keep['transcript_id'] = new_tr['DuplicateTxp'] 

In [263]:
df_keep['count'] = 0.0

In [264]:
df_keep['to_drop'] = None

In [265]:
quant2 = quant.append(df_keep)

In [266]:
quant2.to_csv("/mnt/scratch6/hirak/SLA_benchmark/polyester_bleedthrough/poly.model.60.sf",\
             columns=['transcript_id','count'],sep="\t", index=False)

In [267]:
len(quant2)

200401